### 태양광 발전량 예측 AI 경진대회

안녕하세요 여러분!🙌 태양광 발전량 예측 AI 경진대회에 오신 것을 환영합니다.<br>
태양광 발전은 매일의 기상 상황과 계절에 따른 일사량의 영향을 받습니다. 이에 대한 예측이 가능하다면 보다 원활하게 전력 수급 계획을 세울 수 있습니다.<br>
신재생에너지의 생산 효율성을 극대화하고, 사용자들에게 저렴한 전력을 공급할 수 있도록 인공지능 기반 태양광 발전량 예측 모델을 만들어 주세요.  <br>

모델은 7일(Day 0~ Day6) 동안의 데이터를 인풋으로 활용하여, 향후 2일(Day7 ~ Day8) 동안의 30분 간격의 발전량(TARGET)을 예측해야 합니다. <br>
(1일당 48개씩 총 96개 타임스텝에 대한 예측)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from lightgbm import LGBMRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

# 한글 폰트 패치
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

In [ ]:
HOME = os.getcwd()
trainCSV = os.path.join(HOME, 'data', 'train.csv')
testCSV = os.path.join(HOME, 'data', 'test.csv')

train_df = pd.read_csv(trainCSV)
test_df = pd.read_csv(testCSV)

## 데이터 전처리

In [ ]:
def transform_data(df, input_days=7, output_days=2):
    X = []
    Y = []
    
    # 하루는 48개의 타임스텝 (30분 간격)
    steps_per_day = 48
    input_steps = input_days * steps_per_day
    output_steps = output_days * steps_per_day
    
    # 학습에 사용할 컬럼 정의
    feat_cols = ['DHI', 'DNI', 'WS', 'RH', 'T', 'TARGET']
    
    # 전체 데이터 길이
    total_steps = len(df)
    
    for i in range(0, total_steps - (input_steps + output_steps) + 1, steps_per_day):
        x = df.iloc[i : i + input_steps]
        y = df.iloc[i + input_steps : i + input_steps + output_steps]['TARGET']
        
        # 통계적 요약 피처 생성 (피처 수 대폭 감소)
        x_feats = []
        for col in feat_cols:
            data = x[col].values
            # 전체 기간 통계
            x_feats.extend([
                data.mean(), 
                data.std(), 
                data.min(), 
                data.max(),
                np.percentile(data, 25), 
                np.percentile(data, 75),
                np.median(data)
            ])
            # 일별 평균, 표준편차, 최대값 추가
            for day in range(input_days):
                day_data = data[day * steps_per_day : (day + 1) * steps_per_day]
                x_feats.extend([
                    day_data.mean(),
                    day_data.std(),
                    day_data.max()
                ])
        
        X.append(x_feats)
        Y.append(y.values)
        
    return np.array(X), np.array(Y)

In [ ]:
# 학습 데이터 생성
X, Y = transform_data(train_df)
print(f'Input shape: {X.shape}')
print(f'Target shape: {Y.shape}')

In [ ]:
# 검증셋 분리 (마지막 10% 정도를 검증용으로 사용)
X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size=0.1, shuffle=False, random_state=42)

print(f'Train shape: {X_train.shape}, {y_train.shape}')
print(f'Val shape: {X_val.shape}, {y_val.shape}')

## 모델 학습 (Quantile Regression)

In [ ]:
# Quantile regression models for each quantile
quantiles = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
models = {}

for q in quantiles:
    print(f"Training model for quantile {q}...")
    model = MultiOutputRegressor(
        LGBMRegressor(
            objective='quantile', 
            alpha=q, 
            n_estimators=1000, 
            learning_rate=0.01,
            # 하이퍼파라미터 조정 (고차원 저샘플 문제 해결)
            min_child_samples=5,      # 기본값 20 → 5로 줄임
            min_data_in_leaf=5,       # 리프당 최소 데이터 수
            num_leaves=15,            # 기본값 31 → 15로 줄임
            reg_alpha=0.1,            # L1 정규화
            reg_lambda=0.1,           # L2 정규화
            random_state=42, 
            n_jobs=-1,
            verbose=-1                # 경고 메시지 숨기기
        )
    )
    model.fit(X_train, y_train)
    models[q] = model
    print(f"Model for quantile {q} trained.")

## 결과 예측

In [ ]:
def process_test_data(df):
    X_test = []
    steps_per_day = 48
    days_per_set = 7
    steps_per_set = days_per_set * steps_per_day
    
    # 학습에 사용할 컬럼 정의
    feat_cols = ['DHI', 'DNI', 'WS', 'RH', 'T', 'TARGET']
    
    # test_df의 전체가 Set 단위로 잘림
    num_sets = int(len(df) / steps_per_set)
    
    for i in range(num_sets):
        start_idx = i * steps_per_set
        end_idx = start_idx + steps_per_set
        
        x = df.iloc[start_idx : end_idx]
        
        # 통계적 요약 피처 생성 (train과 동일한 방식)
        x_feats = []
        for col in feat_cols:
            data = x[col].values
            # 전체 기간 통계
            x_feats.extend([
                data.mean(), 
                data.std(), 
                data.min(), 
                data.max(),
                np.percentile(data, 25), 
                np.percentile(data, 75),
                np.median(data)
            ])
            # 일별 평균, 표준편차, 최대값 추가
            for day in range(days_per_set):
                day_data = data[day * steps_per_day : (day + 1) * steps_per_day]
                x_feats.extend([
                    day_data.mean(),
                    day_data.std(),
                    day_data.max()
                ])
        
        X_test.append(x_feats)
        
    return np.array(X_test)

X_test = process_test_data(test_df)
print(f'Test Input shape: {X_test.shape}')

In [ ]:
submission = pd.read_csv('data/sample_submission.csv')

for q in quantiles:
    print(f"Predicting for quantile {q}...")
    # Predict
    pred = models[q].predict(X_test)
    pred = np.maximum(pred, 0) # Ensure non-negative
    
    # Flatten 2D array (samples, 96) to 1D array
    pred_flat = pred.flatten()
    
    # Assign to corresponding column in submission file
    col_name = f'q_{q}'
    submission[col_name] = pred_flat

submission.to_csv('submission.csv', index=False)
print("Submission file 'submission.csv' generated successfully!")

In [ ]:
submission.head()